In [1]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
from reinforcment_based.model.classifier import VGG16
from reinforcment_based.model.evaluator import FaceNet
from reinforcment_based.model.SAC import Agent
from reinforcment_based.model.gan import Generator, Generator_MNIST
from reinforcment_based.utils.mi_attack import inversion_dcgan, inversion_MNIST_DP
from reinforcment_based.utils.file_utils import load_my_state_dict, low2high, seed_everything
import pickle
import warnings
import time

In [2]:
MODEL_NAME = "VGG16"
MAX_EPISODES = 40000
MAX_STEP = 1
SEED = 42
ALPHA = 0
N_CLASSES = 10
Z_DIM = 100
N_TARGET = 10

### load generator trained on MNIST

In [3]:
# G = Generator(Z_DIM)
# path_G = "../training/data/model_data/gan/GeneratorMNISTDCGAN500EPOCHS.tar"
# G = torch.nn.DataParallel(G).cuda()
# G.load_state_dict(torch.load(path_G)['state_dict'], strict=False)
# G.eval()

G = Generator_MNIST(100)
path_G = "checkpoints/gan/Generatormnist.tar"
G = torch.nn.DataParallel(G).cuda()
G.load_state_dict(torch.load(path_G)['state_dict'], strict=False)
G.eval()

DataParallel(
  (module): Generator_MNIST(
    (l1): Sequential(
      (0): Linear(in_features=100, out_features=8192, bias=False)
      (1): BatchNorm1d(8192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (l2_5): Sequential(
      (0): Sequential(
        (0): ConvTranspose2d(512, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): Sequential(
        (0): ConvTranspose2d(256, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (2): Sequential(
        (0): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, m

### load target model

In [4]:
T = VGG16(N_CLASSES)
# path_T = "../training/data/model_data/VGG16/VGG16MNIST.tar"
path_T = "checkpoints/VGG16/dp_VGG16mnist.tar"
T = torch.nn.DataParallel(T).cuda()
T.load_state_dict(torch.load(path_T)['state_dict'], strict=False)
T.eval()

/home/bot/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bot/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


DataParallel(
  (module): VGG16(
    (feature): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=

### load evaluator model

In [5]:
E = VGG16(N_CLASSES)
path_E = 'checkpoints/VGG16/dp_VGG16mnist.tar'
E = torch.nn.DataParallel(E).cuda()
E.load_state_dict(torch.load(path_E)['state_dict'], strict=False)
E.eval()

DataParallel(
  (module): VGG16(
    (feature): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=

In [6]:
seed_everything(seed=SEED)

In [7]:
total = 0
cnt = 0
cnt5 = 0

identities = range(N_CLASSES)
targets = random.sample(identities, N_TARGET)
os.makedirs("attack_results/reinforcment_mnist_dp/images", exist_ok=True)
os.makedirs("attack_results/reinforcment_mnist_dp/models", exist_ok=True)

### Attack Loop

In [8]:
for i in targets:
        start = time.time()
        agent = Agent(state_size=Z_DIM, action_size=Z_DIM, random_seed=SEED, hidden_size=256, action_prior="uniform")
        recon_image = inversion_MNIST_DP(agent, G, T, ALPHA, z_dim=Z_DIM, max_episodes=MAX_EPISODES, max_step=MAX_STEP, label=i, model_name=MODEL_NAME)
        output= E(recon_image)
        eval_prob = F.softmax(output[1], dim=1)
        top_idx = torch.argmax(eval_prob)
        _, top5_idx = torch.topk(eval_prob, 5)

        total += 1
        if top_idx == i:
            cnt += 1
        if i in top5_idx:
            cnt5 += 1

        # prop = T(recon_image)
        # print(prop)

        acc = cnt / total
        acc5 = cnt5 / total
        print("Classes {}/{}, Accuracy : {:.3f}, Top-5 Accuracy : {:.3f}".format(total, N_TARGET, acc, acc5))
        end = time.time()
        total_time = end-start
        print(f"duration for {MAX_EPISODES} episodes: {total_time:.2f}")

Target Label : 1
Episodes 10000/40000, Confidence score for the target model : 0.0951
Episodes 20000/40000, Confidence score for the target model : 0.0952
Episodes 30000/40000, Confidence score for the target model : 0.0952
Episodes 40000/40000, Confidence score for the target model : 0.0954
Classes 1/10, Accuracy : 0.000, Top-5 Accuracy : 1.000
duration for 40000 episodes: 716.85
Target Label : 0
Episodes 10000/40000, Confidence score for the target model : 0.1021
Episodes 20000/40000, Confidence score for the target model : 0.1021
Episodes 30000/40000, Confidence score for the target model : 0.1022
Episodes 40000/40000, Confidence score for the target model : 0.1022
Classes 2/10, Accuracy : 0.000, Top-5 Accuracy : 0.500
duration for 40000 episodes: 708.47
Target Label : 4
Episodes 10000/40000, Confidence score for the target model : 0.0942
Episodes 20000/40000, Confidence score for the target model : 0.0942
Episodes 30000/40000, Confidence score for the target model : 0.0942
Episodes

KeyboardInterrupt: 